# Fashion-MNIST を使ったClassification

fashion-MNIST データセットを使って画像をclassificationを行います

## dataset の確認

今回はオープンデータのfashion-MNISTを使うので、中身を確認します

データの大元はここにあります。
https://github.com/zalandoresearch/fashion-mnist

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

(X_train_orig, y_train_orig), (X_test_orig, y_test_orig) = fashion_mnist.load_data()

## <TODO> それぞれのデータ形を表示してみましょう
print("train feature shape", X_train_orig.___)
print("train label shape", y_train_orig.___)
print("test feature shape", X_test_orig.___)
print("test label shape", y_test_orig.___)

訓練データは60,000件、テストデータは10,000件で、画像のshapeは(28, 28)であることがわかります。縦横しかないからカラー情報しかもたないグレースケール画像であることがわかります。

featureとなる画像データを見てみると様々なファッションアイテムの画像が入っています。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show(n_cols, n_rows, train_orig):
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(16,4))
    for ax, pixels in zip(axs.flat, train_orig):
        ax.imshow(pixels, cmap="gray")
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

show(20, 5, X_train_orig)

labelの意味は https://github.com/zalandoresearch/fashion-mnist#labels ここにあります。

データのそれぞれのカテゴリごとに6,000件ずつ、均等にはいっているようです

In [ ]:
import numpy as np

labels = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'Coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot',
]
left = range(0, 10)
height = np.zeros(10)
for v in y_train_orig:
    height[v] += 1
    
plt.xticks(rotation=45)
plt.bar(left, height, tick_label=labels, align="center")

これでデータについてはわかったので、モデルを作成していきます。

## data preprocessing

モデルを作成するにあたって、データの前処理を行います。
ここでは3つの処理を行っています。
- kerasのcnnで使用するメソッドであるConv2Dは入力のshapeとして(batch_size, rows, cols, channels)を取るため、データをexpandします。
- データの正規化を行います。 (値の範囲を[0-255]から[0-1]にします。)
- ラベルをone hot表現に変換します。('Trouser'が正解の場合、[0,1,0,0,0,0,0,0,0,0]になります。)

In [ ]:
## <TODO> shapeを(batch_size, rows, cols, channels)にexpandする。 (batch_sizeはtrainning時に指定するため、現時点では全データ数で良いです。)
X_train = np.___(_, _)
X_test = np.___(_, _)

print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)

## <TODO> グレースケールの 0-255 の値を 正規化して 0-1 の浮動小数にする
X_train = ___
X_test = ___

## <TODO> one hot vectorにする
y_train = tf.keras.utils.___(y_train_orig, _)
y_test = tf.keras.utils.___(y_test_orig, _)

print("one hot label shape", y_train.shape)

## model

kerasでmodelを作成する場合には2つの方法があります。 Sequential API を使う方法と、Funcional API を使う方法です。

Sequential APIはシンプルで単純な構造のmodelを作る際に便利で、Functional APIは複数の入力やアウトプットをもったり、内部で分岐処理があるような複雑なモデルを作成する際に向いています。
今回はSequential APIを用いてmodelを作ってみましょう

https://www.tensorflow.org/api_docs/python/tf/keras/layers ここからcnnを使ってmodelを組んでみましょう

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

def cnn():
    ## <todo> 資料を参考にモデルを組んでみましょう。
    return model

## training: 訓練

まずモデルを組み立てて、中身を見てみましょう

In [ ]:
%rm -rf ./logs

model = cnn()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

## <todo> モデルをコンパイルする。
model.___(___, ___, metrics=[tf.keras.metrics.CategoricalAccuracy()])
## <todo> 学習を開始する。
model.___(___, ___, ___, ___, ___, callbacks=[tensorboard_callback])

In [ ]:
## <todo> modelの全体像を確認する。
model.___()

## test: 検証

訓練が終わったので、訓練データには存在しないデータで検証をしていきます

In [ ]:
## <todo> modelの検証を行う。
model.___(___, ___)

表示されている情報は lossとcategorical_accuracyです。訓練の結果とどの程度違いがあったでしょうか？ overfitはしていないでしょうか？

## tensorboardでの可視化
また、tensorboardを使うことによって訓練結果の可視化ができます。

In [ ]:
## ./logsをgcsに渡す

USER = "username" # 自分の名前
BUCKET = "mixi-ml-handson-2021"
LOCAL_TRAIN_DIR = "local_train"

!gsutil cp -r ./logs gs://{BUCKET}/{USER}/{LOCAL_TRAIN_DIR}/

In [ ]:
## 出力結果をCloud Shellで実行
print('tensorboard --logdir gs://{}/{}/{}/logs --port 8080'.format(BUCKET, USER, LOCAL_TRAIN_DIR))

上記出力結果をCloud Shellで実行します。

一度cloud consoleのダッシュボードに戻り、右上のアイコンの中の一つにある **Cloud Shellをアクティブにする** をクリックしてください。

cloud Shellが立ち上がったら、出力結果を貼り付けて実行してみてください。

終わったら、cloud shell内右上の歯車マークの右隣にある **ウェブでプレビュー**をクリックし**ポート8080でプレビュー**を選択してください。

これで、学習推移がtensorboardで確認できます。

## prediction: 推論

では出来上がったmodelにリクエストを投げて実際に処理を行ってみましょう

In [ ]:
## <todo> modelから予測結果を受け取る。
predictions = model.___(X_test[10:20])

for i, p in enumerate(predictions):
    print(i, labels[np.argmax(p)], "{}%".format(p[np.argmax(p)]*100))
show(10, 1, X_test_orig[10:20])

## 保存

出来上がったモデルは現時点では、notebookのメモリ上にしかありません。これを保存します。  
save_formatはtensorflow2.0以降のデフォルトの保存形式になっているsaved_model形式で保存を行うための指定です。

In [ ]:
USER    = "username" ## 自分の名前
BUCKET  = "mixi-ml-handson-2021"
VERSION = "001"

## <todo> modelを保存する。
model.___("gs://{}/{}/{}".format(BUCKET, USER, VERSION), save_format="tf")

# 追加課題

+ modelの構成を変更して、どのくらい性能が変わるか確認してみよう
  + Dropoutの有無、正規化の有無、CNNからDNNにした場合
+ 時間に余裕がある場合は 01ex_pruningに進もう